# forex_signal_v27 — PF-first Top‑Down Multi‑Timeframe Ensemble (4H/1H → 15m → 1m)
Энэ notebook нь PF (Profit Factor)‑ийг өсгөх зорилготой **шаталсан (gated) ensemble** систем.

**Гол өөрчлөлт (v26 → v27):**
- Validation дээр threshold‑ийг **жинхэнэ backtest engine**‑ээр (cost+next-open+worst-case) тааруулдаг
- **Prob. calibration** (Platt scaling) → threshold тогтвортой
- Macro gate нь rule‑based дээр нэмээд optional **RandomForest regime score** (soft gate)
- Feature set: BB width, trend/volatility scaling (…/ATR), micro entry filters


In [3]:

# ===== 0) Imports & Global Config =====
import os, glob, re, math, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from dataclasses import dataclass

from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

import xgboost as xgb

SEED = 42
np.random.seed(SEED)

@dataclass
class Config:
    # Paths
    train_dir: str = "../data/train"
    test_dir: str  = "../data/test"

    # Master decision timeframe
    master_tf: str = "15m"  # PF-first default: 15m

    # Timeframes used
    macro_tfs: tuple = ("4h", "1h")
    entry_tfs: tuple = ("1m",)

    # Label / triple-barrier on master TF
    atr_period: int = 14
    tp_atr: float = 2.0
    sl_atr: float = 1.5
    max_hold_bars: int = 16

    # Risk / execution
    initial_equity: float = 10_000.0
    risk_per_trade: float = 0.01
    max_lots: float = 3.0  # safety cap

    # Costs (edit to your broker)
    spread_pips: float = 1.2
    slippage_pips: float = 0.2
    commission_per_lot_usd: float = 0.0

    # FX conventions (EURUSD)
    pip_size: float = 0.0001
    lot_size: float = 100_000
    usd_per_pip_per_lot: float = 10.0  # EURUSD approx

    # Validation split inside train
    train_core_end: str = "2021-12-31"
    val_start: str = "2022-01-01"
    val_end: str = "2023-12-31"

    # Threshold search
    min_trades_val: int = 120
    dd_limit: float = 0.20  # 20% max DD on validation

CFG = Config()
print(CFG)


Config(train_dir='../data/train', test_dir='../data/test', master_tf='15m', macro_tfs=('4h', '1h'), entry_tfs=('1m',), atr_period=14, tp_atr=2.0, sl_atr=1.5, max_hold_bars=16, initial_equity=10000.0, risk_per_trade=0.01, max_lots=3.0, spread_pips=1.2, slippage_pips=0.2, commission_per_lot_usd=0.0, pip_size=0.0001, lot_size=100000, usd_per_pip_per_lot=10.0, train_core_end='2021-12-31', val_start='2022-01-01', val_end='2023-12-31', min_trades_val=120, dd_limit=0.2)


## 1) Data loading
Файл нэршил өөр байж болох тул timeframe string (`1m`, `15m`, `1h`, `4h`)‑ээр хайж олно. CSV дотор дор хаяж: `timestamp, open, high, low, close` байх ёстой.

In [5]:

# ===== 1) Data Loading Helpers =====
REQUIRED_OHLC = ["timestamp","open","high","low","close"]

def _guess_tf_from_filename(fn: str) -> str:
    s = fn.lower()
    # map "m15" -> "15m", "h1" -> "1h"
    # standard
    for tf in ["1m","5m","15m","30m","1h","4h","1d"]:
        if re.search(rf"(?:_|-|\b){tf}(?:_|-|\b)", s):
            return tf
            
    # reversed m15, h4 etc.
    match = re.search(r"(?:_|-|\b)([mh])(\d+)(?:_|-|\b|\.)", s)
    if match:
        unit, val = match.groups()
        return f"{val}{unit}"
    match = re.search(r"(?:_|-|\b)(\d+)([mh])(?:_|-|\b|\.)", s)
    if match:
        val, unit = match.groups()
        return f"{val}{unit}"

    # fallback
    for tf in ["1m","5m","15m","30m","1h","4h","1d"]:
        if tf in s:
            return tf
    return "unknown"

def load_tf_csv(folder: str, tf: str) -> pd.DataFrame:
    files = glob.glob(os.path.join(folder, "*.csv"))
    # first pass: just strict match
    cand = [f for f in files if tf in f.lower()]
    if not cand:
        # try by guessing tf from filename
        cand = [f for f in files if _guess_tf_from_filename(os.path.basename(f)) == tf]
    if not cand:
        raise FileNotFoundError(f"No CSV for tf={tf} in {folder}. Found files={len(files)}")
    # pick the largest file (often the real one)
    cand = sorted(cand, key=lambda p: os.path.getsize(p), reverse=True)
    path = cand[0]
    df = pd.read_csv(path)
    
    # normalize columns
    df.columns = [c.lower() for c in df.columns]
    
    # flexible renaming
    aliases = {
        "timestamp": ["time", "date", "datetime"],
        "open": ["o"],
        "high": ["h"],
        "low": ["l"],
        "close": ["c"]
    }
    
    for std, vars_ in aliases.items():
        if std not in df.columns:
            for v in vars_:
                if v in df.columns:
                    df.rename(columns={v: std}, inplace=True)
                    break
                    
    for c in REQUIRED_OHLC:
        if c not in df.columns:
            raise ValueError(f"Missing column {c} in {path}. Columns={df.columns.tolist()[:20]}")
            
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=False)
    df = df.sort_values("timestamp").drop_duplicates("timestamp")
    df = df.reset_index(drop=True)
    return df

train_master = load_tf_csv(CFG.train_dir, CFG.master_tf)
test_master  = load_tf_csv(CFG.test_dir,  CFG.master_tf)

print("Train master:", train_master.shape, train_master["timestamp"].min(), train_master["timestamp"].max())
print("Test master :", test_master.shape,  test_master["timestamp"].min(),  test_master["timestamp"].max())


Train master: (224382, 6) 2015-01-01 22:00:00 2023-12-29 21:45:00
Test master : (49807, 6) 2024-01-01 22:00:00 2025-12-30 23:45:00


## 2) Technical indicators
PF‑д хэрэгтэй цөөн, давхцахгүй indicator багц:
- Trend: EMA50/EMA200 + slope
- Strength: ADX14
- Volatility: ATR14, BB width
- Momentum: RSI14, MACD hist

Бүх зай/диапазон төрлийн feature‑үүдийг **ATR‑аар нормализац** хийнэ (…/ATR) → volatility regime өөрчлөгдөхөд тогтвортой.

In [6]:

# ===== 2) Indicators =====
def ema(s: pd.Series, span: int) -> pd.Series:
    return s.ewm(span=span, adjust=False).mean()

def rsi(close: pd.Series, period: int = 14) -> pd.Series:
    delta = close.diff()
    up = delta.clip(lower=0).ewm(alpha=1/period, adjust=False).mean()
    down = (-delta.clip(upper=0)).ewm(alpha=1/period, adjust=False).mean()
    rs = up / (down + 1e-12)
    return 100 - (100 / (1 + rs))

def true_range(df: pd.DataFrame) -> pd.Series:
    prev_close = df["close"].shift(1)
    tr = pd.concat([
        (df["high"] - df["low"]),
        (df["high"] - prev_close).abs(),
        (df["low"] - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr

def atr(df: pd.DataFrame, period: int = 14) -> pd.Series:
    return true_range(df).ewm(alpha=1/period, adjust=False).mean()

def adx(df: pd.DataFrame, period: int = 14) -> pd.Series:
    high, low, close = df["high"], df["low"], df["close"]
    plus_dm = (high.diff()).clip(lower=0)
    minus_dm = (-low.diff()).clip(lower=0)
    tr = true_range(df)
    atr_ = tr.ewm(alpha=1/period, adjust=False).mean()
    plus_di = 100 * (plus_dm.ewm(alpha=1/period, adjust=False).mean() / (atr_ + 1e-12))
    minus_di = 100 * (minus_dm.ewm(alpha=1/period, adjust=False).mean() / (atr_ + 1e-12))
    dx = (100 * (plus_di - minus_di).abs() / (plus_di + minus_di + 1e-12))
    return dx.ewm(alpha=1/period, adjust=False).mean()

def macd_hist(close: pd.Series, fast=12, slow=26, signal=9) -> pd.Series:
    macd_line = ema(close, fast) - ema(close, slow)
    sig = ema(macd_line, signal)
    return macd_line - sig

def bb_width(close: pd.Series, period=20, nstd=2.0) -> pd.Series:
    ma = close.rolling(period).mean()
    sd = close.rolling(period).std()
    upper = ma + nstd*sd
    lower = ma - nstd*sd
    width = (upper - lower) / (ma.abs() + 1e-12)
    return width

def add_tf_features(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    out = df.copy()
    out[f"{prefix}_atr14"] = atr(out, CFG.atr_period)
    out[f"{prefix}_ema50"] = ema(out["close"], 50)
    out[f"{prefix}_ema200"] = ema(out["close"], 200)
    out[f"{prefix}_ema200_slope"] = out[f"{prefix}_ema200"].diff(5)
    out[f"{prefix}_rsi14"] = rsi(out["close"], 14)
    out[f"{prefix}_macdh"] = macd_hist(out["close"])
    out[f"{prefix}_adx14"] = adx(out, 14)
    out[f"{prefix}_bbw"] = bb_width(out["close"], 20, 2.0)
    # Normalize distances by ATR
    out[f"{prefix}_dist_ema200_atr"] = (out["close"] - out[f"{prefix}_ema200"]) / (out[f"{prefix}_atr14"] + 1e-12)
    out[f"{prefix}_dist_ema50_atr"]  = (out["close"] - out[f"{prefix}_ema50"]) / (out[f"{prefix}_atr14"] + 1e-12)
    # candle anatomy
    out[f"{prefix}_range_atr"] = (out["high"] - out["low"]) / (out[f"{prefix}_atr14"] + 1e-12)
    out[f"{prefix}_body_atr"]  = (out["close"] - out["open"]).abs() / (out[f"{prefix}_atr14"] + 1e-12)
    return out


## 3) Build multi-timeframe feature table on master TF
- 4H/1H: merge_asof backward (зөвхөн өмнө хаагдсан лаа)
- 1m: 15 минутын цонхоор aggregation (mean/max/std/last)


In [9]:

# ===== 3) Multi-TF Alignment =====
def merge_asof_backward(master: pd.DataFrame, other: pd.DataFrame, other_tf: str, prefix: str) -> pd.DataFrame:
    # ensure other has features
    if prefix not in [c.split('_')[0] for c in other.columns]: # check if already added
        o = add_tf_features(other, prefix)
    else:
        o = other.copy()
    
    cols_to_use = ["timestamp"] + [c for c in o.columns if c.startswith(prefix+"_")]
    o = o[cols_to_use].drop_duplicates("timestamp").sort_values("timestamp")
    
    m = master.sort_values("timestamp")
    merged = pd.merge_asof(m, o, on="timestamp", direction="backward")
    return merged

def agg_entry_1m_to_master(master: pd.DataFrame, one_m: pd.DataFrame, window_minutes: int = 15) -> pd.DataFrame:
    # Optimized Vectorized Implementation
    df = one_m.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.set_index("timestamp").sort_index()
    
    # 1. Pre-calc 1m features
    df["ret1"] = np.log(df["close"]).diff()
    df["rng"] = (df["high"] - df["low"])
    df["body"] = (df["close"] - df["open"]).abs()
    df["wick_down"] = (df[["open","close"]].min(axis=1) - df["low"]).clip(lower=0)
    
    # 2. Rolling aggregation on 1m grid
    # "15min" window, closed='right' means for row at 10:15, include (10:00, 10:15]
    r = df.rolling(window=f"{window_minutes}min", min_periods=5, closed='right')
    
    out = pd.DataFrame(index=df.index)
    out["1m_ret_mean"] = r["ret1"].mean()
    out["1m_ret_std"]  = r["ret1"].std()
    out["1m_ret_sum"]  = r["ret1"].sum()
    out["1m_rng_mean"] = r["rng"].mean()
    out["1m_rng_max"]  = r["rng"].max()
    out["1m_body_mean"] = r["body"].mean()
    out["1m_wick_down_mean"] = r["wick_down"].mean()
    
    # net_move: approximation. Original was last_close - first_close in window.
    # We can approximate with sum of diffs (close - prev_close).
    # Since ret1 is log diff, sum(ret1) is log total return. 
    # Let's just use ret_sum as the feature for "net move" magnitude/direction.
    # It carries similar info.
    out["1m_net_move"] = out["1m_ret_sum"]
    
    out = out.reset_index()
    
    # 3. Join to master
    # master row at T gets 1m rolling stats at T (which cover T-15m..T)
    master = master.sort_values("timestamp")
    merged = pd.merge_asof(master, out, on="timestamp", direction="backward", tolerance=pd.Timedelta("15m")) # tolerance?
    return merged

def build_master_table(folder: str) -> pd.DataFrame:
    master = load_tf_csv(folder, CFG.master_tf)
    master = add_tf_features(master, "m")
    # macro merges
    for tf in CFG.macro_tfs:
        other = load_tf_csv(folder, tf)
        # Note: assuming files exist, if load_tf_csv fails it raises error
        master = merge_asof_backward(master, other, tf, tf)
    # entry 1m agg
    try:
        one_m = load_tf_csv(folder, "1m")
        master = agg_entry_1m_to_master(master, one_m, window_minutes=15)
    except FileNotFoundError:
        print(f"Warning: 1m data not found in {folder}. skipping micro features.")
        # fill micro cols with nan
        for c in ["1m_ret_mean","1m_ret_std","1m_ret_sum","1m_rng_mean","1m_rng_max","1m_body_mean","1m_wick_down_mean","1m_net_move"]:
            master[c] = np.nan
            
    return master

train_df = build_master_table(CFG.train_dir)
test_df  = build_master_table(CFG.test_dir)

print(train_df.shape, test_df.shape)


(224382, 50) (49807, 50)


## 4) Macro gate + optional RF regime score
Rule-based gate PF өсгөхөд хамгийн үр дүнтэй. Үүнийг soft score-оор нэмэгдүүлж болно.


In [10]:

# ===== 4) Macro Gate =====
def macro_gate(row) -> int:
    ok = True
    # 4h: price above EMA200, positive slope, trend strength
    ok &= (row["4h_dist_ema200_atr"] > 0)
    ok &= (row["4h_ema200_slope"] > 0)
    ok &= (row["4h_adx14"] > 18)
    # 1h confirmation
    ok &= (row["1h_dist_ema200_atr"] > -0.25)   # allow mild pullback
    ok &= (row["1h_adx14"] > 16)
    # volatility sanity (avoid extreme)
    ok &= (row["1h_bbw"] > 0.002)               # avoid dead market
    ok &= (row["1h_bbw"] < 0.08)                # avoid explosive chaos
    return int(ok)

for df in (train_df, test_df):
    df["macro_ok"] = df.apply(macro_gate, axis=1)

# Optional: RF regime score trained on TRAIN only (no test leakage)
macro_features = [
    "4h_dist_ema200_atr","4h_ema200_slope","4h_adx14","4h_bbw","4h_range_atr",
    "1h_dist_ema200_atr","1h_ema200_slope","1h_adx14","1h_bbw","1h_range_atr",
]


## 5) Triple-barrier labels on master TF (15m)
Decision: close(t)
Entry: open(t+1)
Barrier evaluation uses future OHLC, но label нь зөвхөн training/validation дээр ашиглагдана.


In [11]:

# ===== 5) Triple-barrier label on master TF =====
def make_triple_barrier_label(df: pd.DataFrame) -> pd.Series:
    atr_ = df["m_atr14"].values
    tp = df["close"].values + CFG.tp_atr * atr_
    sl = df["close"].values - CFG.sl_atr * atr_
    y = np.full(len(df), np.nan)

    for i in range(len(df)):
        tp_i = tp[i]; sl_i = sl[i]
        # start checking from next bar
        end = min(len(df)-1, i + CFG.max_hold_bars)
        hit = None
        for j in range(i+1, end+1):
            hi = df.loc[j, "high"]; lo = df.loc[j, "low"]
            hit_sl = lo <= sl_i
            hit_tp = hi >= tp_i
            if hit_sl or hit_tp:
                # worst-case: if both same bar, count as loss
                if hit_sl and hit_tp:
                    hit = 0
                else:
                    hit = 1 if hit_tp else 0
                break
        if hit is None:
            hit = 0
        y[i] = hit
    return pd.Series(y, index=df.index, name="y_edge")

train_df["y_edge"] = make_triple_barrier_label(train_df)
# test_df label not used for training; keep None


## 6) Train/Validation split (time-based)
Train-core: 2015–2021
Validation: 2022–2023
Final test: 2024–2025


In [15]:

# ===== 6) Split (and pre-calc return features) =====
# Calculate returns BEFORE splitting so they exist in train_core/val
train_df["m_ret1"] = np.log(train_df["close"]).diff()
train_df["m_ret3"] = np.log(train_df["close"]).diff(3)
train_df["m_ret6"] = np.log(train_df["close"]).diff(6)
train_df["m_ret12"]= np.log(train_df["close"]).diff(12)

test_df["m_ret1"] = np.log(test_df["close"]).diff()
test_df["m_ret3"] = np.log(test_df["close"]).diff(3)
test_df["m_ret6"] = np.log(test_df["close"]).diff(6)
test_df["m_ret12"]= np.log(test_df["close"]).diff(12)

train_df = train_df.replace([np.inf,-np.inf], np.nan).ffill()
test_df  = test_df.replace([np.inf,-np.inf], np.nan).ffill()

train_core = train_df[train_df["timestamp"] <= pd.to_datetime(CFG.train_core_end)]
val = train_df[(train_df["timestamp"] >= pd.to_datetime(CFG.val_start)) & (train_df["timestamp"] <= pd.to_datetime(CFG.val_end))]

print("train_core:", train_core.shape, train_core["timestamp"].min(), train_core["timestamp"].max())
print("val       :", val.shape,        val["timestamp"].min(),        val["timestamp"].max())


train_core: (174436, 57) 2015-01-01 22:00:00 2021-12-31 00:00:00
val       : (49859, 57) 2022-01-02 22:00:00 2023-12-29 21:45:00


## 7) Feature sets
### Edge model (15m)
- returns + RSI + MACD + ATR + BB width + distance-to-EMA features + macro context

### Entry model (1m agg)
- micro volatility spike + wick/structure aggregates


In [16]:

# ===== 7) Feature sets =====
edge_features = [
    # master tf
    "m_dist_ema200_atr","m_dist_ema50_atr","m_ema200_slope","m_adx14","m_rsi14","m_macdh","m_bbw","m_range_atr","m_body_atr",
    # simple returns
    # (log returns on master)
]
# Returns already calculated in Split step

edge_features += ["m_ret1","m_ret3","m_ret6","m_ret12"]

# macro context
edge_features += [
    "4h_dist_ema200_atr","4h_ema200_slope","4h_adx14","4h_bbw","4h_range_atr",
    "1h_dist_ema200_atr","1h_ema200_slope","1h_adx14","1h_bbw","1h_range_atr",
]

entry_features = [
    "1m_ret_mean","1m_ret_std","1m_ret_sum","1m_rng_mean","1m_rng_max","1m_body_mean","1m_wick_down_mean","1m_net_move",
    # include current master context to avoid silly entries
    "m_bbw","m_range_atr","m_dist_ema50_atr"
]


## 8) Models
- **Edge:** XGBoost + calibration (Platt)
- **Entry:** LogisticRegression + calibration
- **Optional regime score:** RandomForest (macro features)


In [18]:

# ===== 8) Train models =====
def _prep_xy(df: pd.DataFrame, feats: list, ycol: str):
    d = df.copy()
    d = d.dropna(subset=feats + [ycol, "macro_ok"])
    X = d[feats].values
    y = d[ycol].astype(int).values
    return d, X, y

# --- Optional RF regime score target: use y_edge but only when macro_ok==1 (learn "good regime")
rf_train = train_core.dropna(subset=macro_features + ["y_edge","macro_ok"]).copy()
rf_train = rf_train[rf_train["macro_ok"]==1]
Xr = rf_train[macro_features].values
yr = rf_train["y_edge"].astype(int).values

rf_regime = RandomForestClassifier(
    n_estimators=400,
    max_depth=6,
    min_samples_leaf=40,
    random_state=SEED,
    n_jobs=-1
)
if len(rf_train) > 5000:
    rf_regime.fit(Xr, yr)
    # regime_score = probability of "good trade" given macro state
    train_df.loc[:, "regime_score"] = rf_regime.predict_proba(train_df[macro_features].fillna(method="ffill").values)[:,1]
    test_df.loc[:,  "regime_score"] = rf_regime.predict_proba(test_df[macro_features].fillna(method="ffill").values)[:,1]
else:
    train_df["regime_score"] = 1.0
    test_df["regime_score"]  = 1.0

# --- Edge model (XGB) trained on macro_ok==1 only
edge_train = train_core[(train_core["macro_ok"]==1)].copy()
edge_train, Xe, ye = _prep_xy(edge_train, edge_features, "y_edge")

# imbalance
pos = ye.sum()
neg = len(ye)-pos
scale_pos_weight = (neg/(pos+1e-9)) if pos>0 else 1.0

edge_clf = xgb.XGBClassifier(
    n_estimators=1200,
    max_depth=4,
    learning_rate=0.02,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_weight=6,
    reg_lambda=1.0,
    gamma=0.0,
    random_state=SEED,
    n_jobs=-1,
    eval_metric="logloss",
    scale_pos_weight=scale_pos_weight
)

# time-series CV-ish evaluation (optional)
tscv = TimeSeriesSplit(n_splits=5)
aucs=[]
for tr, te in tscv.split(Xe):
    edge_clf.fit(Xe[tr], ye[tr])
    p = edge_clf.predict_proba(Xe[te])[:,1]
    aucs.append(roc_auc_score(ye[te], p))
print("Edge XGB CV AUC:", np.mean(aucs), "+/-", np.std(aucs))

# Fit final + calibrate on validation slice (from train_df)
edge_clf.fit(Xe, ye)

# Calibration using a held-out part of train (use 2022 as calibrator subset to avoid peeking too much)
cal_df = train_df[(train_df["timestamp"]>=pd.to_datetime("2022-01-01")) & (train_df["timestamp"]<=pd.to_datetime("2022-12-31"))]
cal_df = cal_df[(cal_df["macro_ok"]==1)].dropna(subset=edge_features+["y_edge"])
Xcal = cal_df[edge_features].values
ycal = cal_df["y_edge"].astype(int).values

class ManualCalibratedClassifier:
    def __init__(self, base_estimator):
        self.base_estimator = base_estimator
        self.lr = LogisticRegression(C=9999999)
    def fit(self, X, y):
        preds = self.base_estimator.predict_proba(X)[:, 1].reshape(-1, 1)
        self.lr.fit(preds, y)
    def predict_proba(self, X):
        preds = self.base_estimator.predict_proba(X)[:, 1].reshape(-1, 1)
        return self.lr.predict_proba(preds)

try:
    edge_cal = CalibratedClassifierCV(edge_clf, method="sigmoid", cv="prefit")
    if len(cal_df) > 1000:
        edge_cal.fit(Xcal, ycal)
    else:
        edge_cal = edge_clf
except Exception as e:
    print(f"Edge cal failed: {e}. Using manual.")
    if len(cal_df) > 1000:
        edge_cal = ManualCalibratedClassifier(edge_clf)
        edge_cal.fit(Xcal, ycal)
    else:
        edge_cal = edge_clf

# --- Entry model: label = avoid immediate adverse move proxy (on train_core)
# Create a simple adverse-move label on master TF using next 3 bars low vs next-open entry.
def make_entry_label(df: pd.DataFrame, lookahead_bars: int = 3, adverse_atr: float = 0.5) -> pd.Series:
    atr_ = df["m_atr14"].values
    y = np.full(len(df), np.nan)
    for i in range(len(df)-lookahead_bars-1):
        entry = df.loc[i+1, "open"]  # next-open
        future_low = df.loc[i+1:i+lookahead_bars, "low"].min()
        adverse = (entry - future_low) / (atr_[i] + 1e-12)
        y[i] = 1 if adverse < adverse_atr else 0   # 1 = clean entry (not too adverse)
    return pd.Series(y, index=df.index, name="y_entry")

train_df["y_entry"] = make_entry_label(train_df)
# Need to update train_core entries or recreate
# Since we modified train_df, let's just re-slice train_core logic for entry training
# train_core was defined by timestamp.
entry_train_df = train_df[train_df["timestamp"] <= pd.to_datetime(CFG.train_core_end)]
entry_train = entry_train_df[(entry_train_df["macro_ok"]==1)].dropna(subset=entry_features+["y_entry"]).copy()
Xen = entry_train[entry_features].values
yen = entry_train["y_entry"].astype(int).values

entry_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=2000, C=1.0, class_weight="balanced", random_state=SEED))
])

# calibrate entry probabilities too
entry_pipe.fit(Xen, yen)

# calibrate on 2022 subset
cal2 = train_df[(train_df["timestamp"]>=pd.to_datetime("2022-01-01")) & (train_df["timestamp"]<=pd.to_datetime("2022-12-31"))]
cal2 = cal2[(cal2["macro_ok"]==1)].dropna(subset=entry_features+["y_entry"])

try:
    entry_cal = CalibratedClassifierCV(entry_pipe, method="sigmoid", cv="prefit")
    if len(cal2) > 1000:
        entry_cal.fit(cal2[entry_features].values, cal2["y_entry"].astype(int).values)
    else:
        entry_cal = entry_pipe
except Exception as e:
    print(f"Entry cal failed: {e}. Using manual.")
    if len(cal2) > 1000:
        entry_cal = ManualCalibratedClassifier(entry_pipe)
        entry_cal.fit(cal2[entry_features].values, cal2["y_entry"].astype(int).values)
    else:
        entry_cal = entry_pipe

print("Models ready.")


Edge XGB CV AUC: 0.6438650508750831 +/- 0.029016944185844856
Edge cal failed: The 'cv' parameter of CalibratedClassifierCV must be an int in the range [2, inf), an object implementing 'split' and 'get_n_splits', an iterable or None. Got 'prefit' instead.. Using manual.
Entry cal failed: The 'cv' parameter of CalibratedClassifierCV must be an int in the range [2, inf), an object implementing 'split' and 'get_n_splits', an iterable or None. Got 'prefit' instead.. Using manual.
Models ready.


## 9) Backtest engine (үнэн зөв)
Дүрэм:
- signal at close(t)
- entry at open(t+1)
- long entry uses ask (mid + spread/2 + slip)
- exit uses bid (mid - spread/2 - slip)
- TP/SL нэг лаанд зэрэг хүрвэл worst-case → SL гэж үзнэ


In [19]:

# ===== 9) Backtest Engine =====
@dataclass
class Trade:
    entry_time: pd.Timestamp
    exit_time: pd.Timestamp
    entry_price: float
    exit_price: float
    sl: float
    tp: float
    lots: float
    pnl_usd: float
    pnl_pips: float
    outcome: str

def pip_to_price(pips: float) -> float:
    return pips * CFG.pip_size

def price_to_pips(delta_price: float) -> float:
    return delta_price / CFG.pip_size

def calc_lots_for_risk(equity: float, stop_pips: float) -> float:
    if stop_pips <= 0:
        return 0.0
    risk_usd = equity * CFG.risk_per_trade
    lots = risk_usd / (stop_pips * CFG.usd_per_pip_per_lot)
    return float(max(0.0, min(lots, CFG.max_lots)))

def run_backtest(df: pd.DataFrame, signal: np.ndarray) -> tuple[list, pd.DataFrame]:
    spread = pip_to_price(CFG.spread_pips)
    slip = pip_to_price(CFG.slippage_pips)

    equity = CFG.initial_equity
    eq = []
    trades = []

    in_pos = False
    entry_i = None
    entry_price = sl = tp = lots = None

    for i in range(len(df)-1):
        t = df.loc[i, "timestamp"]
        eq.append((t, equity))

        if not in_pos:
            if signal[i] == 1:
                # decide at close(i), enter at open(i+1) using ask
                entry_mid = df.loc[i+1, "open"]
                entry_price = entry_mid + spread/2 + slip
                # stop/tp based on ATR at decision bar i
                atr_i = df.loc[i, "m_atr14"]
                sl = entry_mid - CFG.sl_atr * atr_i
                tp = entry_mid + CFG.tp_atr * atr_i
                stop_pips = price_to_pips(entry_price - (sl + spread/2 + slip))  # worst-ish
                lots = calc_lots_for_risk(equity, stop_pips)
                if lots > 0:
                    in_pos = True
                    entry_i = i+1
        else:
            hi = df.loc[i, "high"]
            lo = df.loc[i, "low"]

            hit_sl = lo <= sl
            hit_tp = hi >= tp

            if hit_sl or hit_tp:
                if hit_sl and hit_tp:
                    exit_mid = sl
                    outcome = "SL(ambiguous)"
                elif hit_sl:
                    exit_mid = sl
                    outcome = "SL"
                else:
                    exit_mid = tp
                    outcome = "TP"

                # long exit uses bid
                exit_price = exit_mid - spread/2 - slip
                pnl_price = exit_price - entry_price
                pnl_pips = price_to_pips(pnl_price)
                pnl_usd = pnl_pips * CFG.usd_per_pip_per_lot * lots
                pnl_usd -= CFG.commission_per_lot_usd * lots

                equity += pnl_usd
                trades.append(Trade(
                    entry_time=df.loc[entry_i, "timestamp"],
                    exit_time=t,
                    entry_price=float(entry_price),
                    exit_price=float(exit_price),
                    sl=float(sl), tp=float(tp),
                    lots=float(lots),
                    pnl_usd=float(pnl_usd),
                    pnl_pips=float(pnl_pips),
                    outcome=outcome
                ))
                in_pos = False
                entry_i = None

    eq_df = pd.DataFrame(eq, columns=["timestamp","equity"])
    eq_df["peak"] = eq_df["equity"].cummax()
    eq_df["dd"] = (eq_df["equity"] - eq_df["peak"]) / eq_df["peak"]
    return trades, eq_df

def summarize(trades: list[Trade], eq_df: pd.DataFrame) -> dict:
    if not trades:
        return {"trades":0, "profit_factor":0.0, "winrate":0.0, "net_profit_usd":0.0, "max_drawdown":0.0}
    pnl = np.array([t.pnl_usd for t in trades])
    gp = pnl[pnl>0].sum()
    gl = -pnl[pnl<0].sum()
    pf = gp/gl if gl>0 else float("inf")
    return {
        "trades": int(len(trades)),
        "profit_factor": float(pf),
        "winrate": float((pnl>0).mean()),
        "net_profit_usd": float(pnl.sum()),
        "max_drawdown": float(eq_df["dd"].min()) if len(eq_df) else 0.0,
        "avg_trade_usd": float(pnl.mean()),
    }


## 10) Threshold tuning on validation (жинхэнэ backtest)
Энд PF‑ийг *жинхэнэ engine*-ээр maximize хийнэ. Шүүлт:
- trades >= `min_trades_val`
- maxDD <= 20%

Дараа нь хамгийн сайн 1–3 тохиргооноос сонгоод TEST дээр нэг л удаа үнэлнэ.

In [20]:

# ===== 10) Validation threshold tuning with real backtest =====
def compute_probs(df: pd.DataFrame):
    d = df.copy()
    # edge probs only meaningful when macro_ok=1; else set 0
    mask = (d["macro_ok"].values==1)
    p_edge = np.zeros(len(d))
    p_entry = np.zeros(len(d))
    if mask.sum() > 0:
        xe = d.loc[mask, edge_features].fillna(method="ffill").values
        xn = d.loc[mask, entry_features].fillna(method="ffill").values
        pe = edge_cal.predict_proba(xe)[:,1]
        pn = entry_cal.predict_proba(xn)[:,1]
        p_edge[mask] = pe
        p_entry[mask] = pn
    # regime score as soft gate (0..1)
    if "regime_score" in d.columns:
        rs = d["regime_score"].fillna(1.0).values
    else:
        rs = np.ones(len(d))
    return p_edge, p_entry, rs

val_use = val.copy().reset_index(drop=True)
pve, pvn, rsv = compute_probs(val_use)

def make_signal(p_edge, p_entry, regime_score, edge_th, entry_th, regime_th):
    sig = (val_use["macro_ok"].values==1) & (p_edge>edge_th) & (p_entry>entry_th) & (regime_score>regime_th)
    return sig.astype(int)

edge_grid  = np.round(np.linspace(0.58, 0.78, 11), 2)
entry_grid = np.round(np.linspace(0.52, 0.72, 11), 2)
reg_grid   = np.round(np.linspace(0.45, 0.75, 7), 2)

best = None
results=[]
for eth in edge_grid:
    for ith in entry_grid:
        for rth in reg_grid:
            sig = make_signal(pve, pvn, rsv, eth, ith, rth)
            trades, eq = run_backtest(val_use, sig)
            s = summarize(trades, eq)
            if s["trades"] < CFG.min_trades_val:
                continue
            if abs(s["max_drawdown"]) > CFG.dd_limit:
                continue
            results.append((eth, ith, rth, s["profit_factor"], s["trades"], s["net_profit_usd"], s["max_drawdown"]))
            if (best is None) or (s["profit_factor"] > best[3]):
                best = (eth, ith, rth, s["profit_factor"], s["trades"], s["net_profit_usd"], s["max_drawdown"])

res_df = pd.DataFrame(results, columns=["edge_th","entry_th","regime_th","pf","trades","net_usd","max_dd"])
print("Candidates:", len(res_df))
if len(res_df):
    print("Top 10 by PF:")
    display(res_df.sort_values("pf", ascending=False).head(10))
print("BEST:", best)

EDGE_TH, ENTRY_TH, REG_TH = (best[0], best[1], best[2]) if best else (0.65, 0.58, 0.55)
print("Using thresholds:", EDGE_TH, ENTRY_TH, REG_TH)


Candidates: 0
BEST: None
Using thresholds: 0.65 0.58 0.55


## 11) Final backtest on TEST (2024–2025)
TEST дээр threshold‑ийг дахиж тааруулахгүй. Зөвхөн нэг удаа үнэл.

In [21]:

# ===== 11) Final TEST backtest =====
test_use = test_df.copy().reset_index(drop=True)
pte, ptn, rst = compute_probs(test_use)

sig_test = ((test_use["macro_ok"].values==1) & (pte>EDGE_TH) & (ptn>ENTRY_TH) & (rst>REG_TH)).astype(int)
trades, eq = run_backtest(test_use, sig_test)
summary = summarize(trades, eq)
print("TEST summary:", summary)

trades_df = pd.DataFrame([t.__dict__ for t in trades]) if trades else pd.DataFrame()
if len(trades_df):
    trades_df["year"] = pd.to_datetime(trades_df["exit_time"]).dt.year
    print("\nTrades per year:")
    print(trades_df.groupby("year")["pnl_usd"].agg(["count","sum","mean"]))


TEST summary: {'trades': 0, 'profit_factor': 0.0, 'winrate': 0.0, 'net_profit_usd': 0.0, 'max_drawdown': 0.0}


## 12) Next steps to raise PF further
Хэрвээ PF бага байвал хамгийн үр дүнтэй 3 knob:
1) **EDGE_TH өсгөх** (selectivity ↑ → PF ↑, trades ↓)
2) Macro gate‑ийг чангаруулах (ADX/BB width/EMA slope)
3) TP/SL харьцаа (TP=2.2×ATR, SL=1.4×ATR гэх мэт) — validation дээр л тааруул
